In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('train.csv', index_col='Id')

In [25]:

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)
# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [26]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


# Step 1: Preliminary investigation

In [27]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [28]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Step 2: Drop columns with missing values
### In this step, you'll preprocess the data in `X_train` and `X_valid` to remove columns with missing values.  Set the preprocessed DataFrames to `reduced_X_train` and `reduced_X_valid`, respectively.  

In [29]:
# Fill in the line below: get names of columns with missing values
# Your code here
cols_with_missing= [col for col in X_train.columns
                    if X_train[col].isnull().any()]

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing,axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing,axis=1)

In [30]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


# Step 3: Imputation

### Part A

### next code cell to impute missing values with the mean value along each column.  Set the preprocessed DataFrames to `imputed_X_train` and `imputed_X_valid`.  Make sure that the column names match those in `X_train` and `X_valid`.

In [31]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
# Your code here
my_imputer=SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


In [32]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


# Step 4: Generate test predictions
### in this step you will generate preprocessed the training and validation features, you'll train and evaluate a random forest model.  Then, you'll preprocess the test data before generating predictions 

In [33]:
# Step 1: Preliminary investigation
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [34]:
# This columns looks like has numbers on which aggrigate operations can be performed
X_train['LotFrontage']

Id
619      90.0
871      60.0
93       80.0
818       NaN
303     118.0
        ...  
764      82.0
836      60.0
1217     68.0
560       NaN
685      58.0
Name: LotFrontage, Length: 1168, dtype: float64

In [35]:
# Not sure of this columns as there are 0's as wll as null columns which will divert the results so better to remove this column
X_train[X_train['MasVnrArea']<1]

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
1455,20,62.0,7500,7,5,2004,2005,0.0,410,0,...,400,0,113,0,0,0,0,0,10,2009
960,160,24.0,2572,7,5,1999,1999,0.0,604,0,...,484,0,44,0,0,0,0,0,5,2010
76,180,21.0,1596,4,5,1973,1973,0.0,462,0,...,297,120,101,0,0,0,0,0,11,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,160,24.0,1950,6,6,1980,1980,0.0,81,612,...,452,161,0,0,0,0,0,0,7,2008
278,20,140.0,19138,4,5,1951,1951,0.0,120,0,...,400,0,0,0,0,0,0,0,6,2010
1384,30,NaN,25339,5,7,1918,2007,0.0,0,0,...,576,0,0,112,0,0,0,0,8,2007


In [36]:
# we may not able to perform agrigate opperatins on the year but we can take a meadian and give an extantion to imputation
X_train['GarageYrBlt']

Id
619     2007.0
871     1962.0
93      1921.0
818     2002.0
303     2001.0
         ...  
764     1999.0
836     1996.0
1217    1978.0
560     2003.0
685     1998.0
Name: GarageYrBlt, Length: 1168, dtype: float64

In [37]:
# Preprocessed training and validation features
drop_X_train = X_train.drop('MasVnrArea',axis=1)
drop_X_valid = X_valid.drop('MasVnrArea',axis=1)
final_imputed_X_train = drop_X_train.copy()
final_imputed_X_valid = drop_X_valid.copy()

final_imputed_X_train['GarageYrBlt'+'_was_missing']=final_imputed_X_train['GarageYrBlt'].isnull()
final_imputed_X_valid['GarageYrBlt'+'_was_missing']=final_imputed_X_valid['GarageYrBlt'].isnull()
    
final_imputer=SimpleImputer(strategy='median')
final_X_train=pd.DataFrame(final_imputer.fit_transform(final_imputed_X_train))
final_X_valid=pd.DataFrame(final_imputer.transform(final_imputed_X_valid))

final_X_train.columns=final_imputed_X_train.columns
final_X_valid.columns=final_imputed_X_valid.columns


In [38]:
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17788.552283105022
